In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/weighted-average-pool/output.log
/kaggle/input/weighted-average-pool/output/oof_df.csv
/kaggle/input/weighted-average-pool/output/microsoft_deberta-v3-base_fold_1_best.pth
/kaggle/input/weighted-average-pool/output/microsoft_deberta-v3-base_fold_4_best.pth
/kaggle/input/weighted-average-pool/output/config.pth
/kaggle/input/weighted-average-pool/output/microsoft_deberta-v3-base_fold_3_best.pth
/kaggle/input/weighted-average-pool/output/microsoft_deberta-v3-base_fold_2_best.pth
/kaggle/input/weighted-average-pool/output/microsoft_deberta-v3-base_fold_0_best.pth
/kaggle/input/weighted-average-pool/output/tokenizer/spm.model
/kaggle/input/weighted-average-pool/output/tokenizer/tokenizer.json
/kaggle/input/weighted-average-pool/output/tokenizer/tokenizer_config.json
/kaggle/input/weighted-average-pool/output/tokenizer/special_tokens_map.json
/kaggle/input/weighted-average-pool/output/tokenizer/added_tokens.json
/kaggle/input/first-run-v1/output.log
/kaggle/input/first-run-v1/o

In [2]:
import ast
import copy
import gc
import itertools
import joblib
import json
import math
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import os
import pandas as pd
import pickle
import random
import re
import scipy as sp
import string
import sys
import time
import torch
import torch.nn as nn
import warnings
from sklearn.metrics import cohen_kappa_score
from torch.nn import Parameter
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm
# ======= OPTIONS =========
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Current device is: {device}")
warnings.filterwarnings("ignore")
!mkdir output

Current device is: cuda


In [3]:
import tokenizers
import transformers
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
%env TOKENIZERS_PARALLELISM=true
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")

env: TOKENIZERS_PARALLELISM=true
tokenizers.__version__: 0.15.2
transformers.__version__: 4.39.3


In [4]:
class config:
    APEX = True # Automatic Precision Enabled
    BATCH_SIZE_TEST = 16
    DEBUG = False
    GRADIENT_CHECKPOINTING = True
    MAX_LEN = 512
    MODEL = "microsoft/deberta-v3-base"
    NUM_CLASSES = 6
    NUM_WORKERS = 0 #multiprocessing.cpu_count()
    PRINT_FREQ = 20
    SEED = 20
class paths:
    BEST_MODEL_PATH = "/kaggle/input/weighted-average-pool/output"
    TokenizerPath= "/kaggle/input/first-run-v1/output"
    OUTPUT_DIR = "/content/output"
    SUBMISSION_CSV = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv"
    TEST_CSV = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv"
    TRAIN_CSV = "/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv"

In [5]:
def get_score(y_true, y_pred):
    score = cohen_kappa_score(y_true, y_pred, weights='quadratic')
    return score
def seed_everything(seed=20):
    """Seed everything to ensure reproducibility"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
def sep():
    print("-"*100)
seed_everything(seed=config.SEED)

In [6]:
test_df = pd.read_csv(paths.TEST_CSV, sep=',')
print(f"Test dataframe has shape: {test_df.shape}"), sep()
display(test_df.head())

Test dataframe has shape: (3, 2)
----------------------------------------------------------------------------------------------------


,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [7]:
tokenizer = AutoTokenizer.from_pretrained(paths.TokenizerPath + "/tokenizer")
vocabulary = tokenizer.get_vocab()
total_tokens = len(vocabulary)
print("Total number of tokens in the tokenizer:", total_tokens)
print(tokenizer)

Total number of tokens in the tokenizer: 128001
DebertaV2TokenizerFast(name_or_path='/kaggle/input/first-run-v1/output/tokenizer', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=Fal

In [8]:
def prepare_input(cfg, text, tokenizer):
    """
    This function tokenizes the input text with the configured padding and truncation. Then,
    returns the input dictionary, which contains the following keys: "input_ids",
    "token_type_ids" and "attention_mask". Each value is a torch.tensor.
    :param cfg: configuration class with a TOKENIZER attribute.
    :param text: a numpy array where each value is a text as string.
    :return inputs: python dictionary where values are torch tensors.
    """
    inputs = tokenizer.encode_plus(
        text,
        return_tensors=None,
        add_special_tokens=True,
        max_length=cfg.MAX_LEN,
        padding='max_length', # TODO: check padding to max sequence in batch
        truncation=True
    )
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long) # TODO: check dtypes
    return inputs
def collate(inputs):
    """
    It truncates the inputs to the maximum sequence length in the batch.
    """
    mask_len = int(inputs["attention_mask"].sum(axis=1).max()) # Get batch's max sequence length
    for k, v in inputs.items():
        inputs[k] = inputs[k][:,:mask_len]
    return inputs
class CustomDataset(Dataset):
    def __init__(self, cfg, df, tokenizer):
        self.cfg = cfg
        self.tokenizer = tokenizer
        self.texts = df['full_text'].values
        self.essay_ids = df['essay_id'].values
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, item):
        output = {}
        output["inputs"] = prepare_input(self.cfg, self.texts[item], self.tokenizer)
        output["essay_ids"] = self.essay_ids[item]
        return output

In [9]:
class MeanPooling(nn.Module):
    def __init__(self):
        super(MeanPooling, self).__init__()
    def forward(self, last_hidden_state, attention_mask):
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
        sum_embeddings = torch.sum(last_hidden_state * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        sum_mask = torch.clamp(sum_mask, min=1e-9)
        mean_embeddings = sum_embeddings / sum_mask
        return mean_embeddings
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        # Load config by inferencing it from the model name.
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.MODEL, output_hidden_states=True)
            self.config.hidden_dropout = 0.
            self.config.hidden_dropout_prob = 0.
            self.config.attention_dropout = 0.
            self.config.attention_probs_dropout_prob = 0.
        # Load config from a file.
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.MODEL, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        if self.cfg.GRADIENT_CHECKPOINTING:
            self.model.gradient_checkpointing_enable()
        # Add MeanPooling and Linear head at the end to transform the Model into a RegressionModel
        self.pool = MeanPooling()
        self.fc = nn.Linear(self.config.hidden_size, config.NUM_CLASSES)
    def _init_weights(self, module):
        """
        This method initializes weights for different types of layers. The type of layers
        supported are nn.Linear, nn.Embedding and nn.LayerNorm.
        """
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)
    def feature(self, inputs):
        """
        This method makes a forward pass through the model, get the last hidden state (embedding)
        and pass it through the MeanPooling layer.
        """
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        feature = self.pool(last_hidden_states, inputs['attention_mask'])
        return feature
    def forward(self, inputs):
        """
        This method makes a forward pass through the model, the MeanPooling layer and finally
        then through the Linear layer to get a regression value.
        """
        feature = self.feature(inputs)
        output = self.fc(feature)
        return output

In [10]:
def inference_fn(config, test_df, tokenizer, device):
    # ======== DATASETS ==========
    test_dataset = CustomDataset(config, test_df, tokenizer)
    # ======== DATALOADERS ==========
    test_loader = DataLoader(
        test_dataset,
        batch_size=config.BATCH_SIZE_TEST,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False
    )
    
    # ======== MODEL ==========
    softmax = nn.Softmax(dim=1)
    model = CustomModel(config, config_path=paths.BEST_MODEL_PATH + "/config.pth", pretrained=False)
    state = torch.load(paths.BEST_MODEL_PATH + "/microsoft_deberta-v3-base_fold_3_best.pth")
    model.load_state_dict(state["model"])
    model.to(device)
    model.eval() # set model in evaluation mode
    prediction_dict = {}
    preds = []
    idx = []
    with tqdm(test_loader, unit="test_batch", desc='Inference') as tqdm_test_loader:
        for step, batch in enumerate(tqdm_test_loader):
            inputs = batch.pop("inputs")
            ids = batch.pop("essay_ids")
            inputs = collate(inputs) # collate inputs
            for k, v in inputs.items():
                inputs[k] = v.to(device) # send inputs to device
            with torch.no_grad():
                y_preds = model(inputs) # forward propagation pass
                _, y_preds = torch.max(softmax(torch.tensor(y_preds)), dim=1)
            preds.append(y_preds.to('cpu').numpy()) # save predictions
            idx.append(ids)
    prediction_dict["predictions"] = np.concatenate(preds)
    prediction_dict["essay_ids"] = np.concatenate(idx)
    return prediction_dict

In [11]:
predictions = inference_fn(config, test_df, tokenizer, device)

Inference:   0%|          | 0/1 [00:00<?, ?test_batch/s]

In [12]:
submission = pd.DataFrame()
submission["essay_id"] = predictions["essay_ids"]
submission["score"] = predictions["predictions"]
submission["score"] = submission["score"] + 1 
print(f"Submission shape: {submission.shape}")
submission.to_csv("submission.csv", index=False)
submission

Submission shape: (3, 2)


,essay_id,score
0,000d118,2
1,000fe60,2
2,001ab80,4
